# Accessing BLS data

There is at least one type that required some special attention in a BLS file as it was not listed in the `xAOD` EDM. To fix this, there is now a [special file](https://github.com/gordonwatts/func-adl-types-atlas/blob/main/scripts/required_classes.txt) that contains a list of `extra` types.

This demo shows how to use those classes.

## Local ServiceX Against a file

This file was taken from the dataset `user.ponyisi:user.ponyisi.mc23_13p6TeV.524546.MGPy8EG_23lo_S4b18p4NJ0_Upsi1S2mu_4mu_3pt2_v2.deriv.DAOD_BPHY4.e8548_a911_r14908_p5923`. Any BLS file should work.

In [1]:
from pathlib import Path
from servicex import dataset



file = Path("DAOD_BPHY4.35498368._000001.pool.root.1").absolute()

assert file.exists()

ds = dataset.FileList([str(file)])

Next, lets create the local sx instance (for testing).

In [2]:
from servicex_local import (
    LocalXAODCodegen,
    SXLocalAdaptor,
    WSL2ScienceImage,
    DockerScienceImage
)
from servicex.configuration import Configuration, Endpoint
from servicex_local.adaptor import MinioLocalAdaptor

codegen_name = "atlasr22-local"
backend_name = "local-backend"

codegen = LocalXAODCodegen()
# science_runner = WSL2ScienceImage("atlas_al9", "25.2.12")
science_runner = DockerScienceImage(
    "sslhep/servicex_func_adl_xaod_transformer:22.2.107"
)
adaptor = SXLocalAdaptor(codegen, science_runner, codegen_name, "http://localhost:5001")

Configuration.register_endpoint(
    Endpoint(
        name=backend_name,
        adapter=adaptor,
        minio=MinioLocalAdaptor.for_transform,  # type: ignore
        endpoint="bogus",
    )
)

## The BLS query that fetches just the orphaned type data

We have to declare the type first.

In [3]:
from func_adl_servicex_xaodr22 import FuncADLQueryPHYSLITE

query = (
    FuncADLQueryPHYSLITE()
    .Select(lambda e: e.Vertices("BPHY4Quads"))
    .Select(
        lambda vtxs: {
            "x": vtxs.Select(lambda v: v.x()),
            # "QUAD_Muon0": vtxs.Select(lambda v: v.auxdataConst("MuonLinks")[0].pt())
        }
    )
)

Next fetch the data

In [4]:
import logging

logging.basicConfig(level=logging.INFO)

In [14]:
from servicex import deliver

request = {
    "Sample": [
        {
            "Name": "TestBLSQuadMu0",
            "Dataset": ds,
            "Query": query,
            "Codegen": "atlasr22-local",
            "IgnoreLocalCache": True,
        }
    ]
}

result = deliver(request, servicex_name=backend_name)

INFO:servicex.servicex_client:Returning code generators from cache


Output()

In [15]:
result['TestBLSQuadMu0'][0]

ReturnValueException: Exception occurred while making ServiceX request.
Traceback (most recent call last):
  File "c:\Users\gordo\Code\iris-hep\bls_example\.venv\Lib\site-packages\servicex_local\science_images.py", line 176, in transform
    subprocess.run(
    ~~~~~~~~~~~~~~^
        [
        ^
    ...<18 lines>...
        stderr=subprocess.PIPE,
        ^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\Users\gordo\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 577, in run
    raise CalledProcessError(retcode, process.args,
                             output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['docker', 'run', '--name', 'sx_codegen_container_sslhep_servicex_func_adl_xaod_transformer_22.2.107_DAOD_BPHY4.35498368._000001.pool.root', '--rm', '-v', 'C:\\Users\\gordo\\AppData\\Local\\Temp\\tmpdixdjxa5:/generated', '-v', 'C:\\Users\\gordo\\AppData\\Local\\Temp\\servicex\\9481a1d9-30a3-4878-a340-37d239907ce9:/servicex/output', 'sslhep/servicex_fun

Old code we are working from

```python

from servicex import deliver, query as q, dataset as d
import uproot
import hist
import mplhep
import awkward as ak
import vector
from func_adl_servicex_xaodr22 import (
    FuncADLQueryPHYSLITE,
    cpp_float,
    cpp_string,
    tdt_chain_fired,
    calib_tools,
    cpp_vfloat,
)

DS = "user.ponyisi:user.ponyisi.mc23_13p6TeV.524546.MGPy8EG_23lo_S4b18p4NJ0_Upsi1S2mu_4mu_3pt2_v2.deriv.DAOD_BPHY4.e8548_a911_r14908_p5923"

vector.register_awkward()

query = FuncADLQueryPHYSLITE()

# query = calib_tools.query_update(query, muon_working_point='Medium')
query = query.Select(
    lambda e: {
        "evt": e.EventInfo("EventInfo"),
        "quads": e.Vertices("BPHY4Quads"),
        "muons": e.Muons("Muons"),
    }
)
query = query.Select(
    lambda e: {
        "eventNumber": e.evt.eventNumber(),
        "QUAD_mass": e.quads.Select(lambda q: q.auxdataConst[cpp_float]("QUAD_mass")),
        "QUAD_ChargeCode": e.quads.Select(
            lambda q: q.auxdataConst[cpp_string]("ChargeCode")
        ),
        "QUAD_CombinationCode": e.quads.Select(
            lambda q: q.auxdataConst[cpp_string]("CombinationCode")
        ),
        "QUAD_RefTrackPx": e.quads.Select(
            lambda q: q.auxdataConst[cpp_vfloat]("RefTrackPx")
        ),
        "QUAD_RefTrackPy": e.quads.Select(
            lambda q: q.auxdataConst[cpp_vfloat]("RefTrackPy")
        ),
        "QUAD_RefTrackPz": e.quads.Select(
            lambda q: q.auxdataConst[cpp_vfloat]("RefTrackPz")
        ),
        "QUAD_chi2": e.quads.Select(lambda q: q.chiSquared()),
        "QUAD_dof": e.quads.Select(lambda q: q.numberDoF()),
        "mu_pt": e.muons.Select(lambda q: q.pt()),
        "mu_eta": e.muons.Select(lambda q: q.eta()),
        "mu_phi": e.muons.Select(lambda q: q.phi()),
        # "QUAD_Muon0": e.quads.Select(lambda q: q.auxdataConst["std::vector<ElementLink<xAOD::MuonContainer>>"]("MuonLinks")[0].pt()),
        "HLT_mu4_ivarloose_mu4_mu3noL1": tdt_chain_fired(
            "HLT_mu4_ivarloose_mu4_mu3noL1_L1BPH-7M14-0DR25-MU5VFMU3VF"
        ),
        "HLT_2mu4_ivarloose": tdt_chain_fired(
            "HLT_2mu4_ivarloose_L1BPH-7M14-0DR25-MU5VFMU3VF"
        ),
        "HLT_2mu4": tdt_chain_fired("HLT_2mu4_L1BPH-7M11-25DR99-2MU3VF"),
        "HLT_3mu4_bUpsi": tdt_chain_fired("HLT_3mu4_bUpsi_L13MU3VF"),
        "HLT_2mu6_bUpsimumu": tdt_chain_fired(
            "HLT_2mu6_bUpsimumu_L1BPH-8M15-0DR22-2MU5VF"
        ),
        "HLT_mu11_mu6_bDimu": tdt_chain_fired("HLT_mu11_mu6_bDimu_L1MU8VF_2MU5VF"),
    }
)

dssplit = DS.split(".")
dsname = f"bphy4.{dssplit[4]}.{dssplit[8]}"

request = {
    "Sample": [
        {
            "Name": dsname,
            "Dataset": d.Rucio(DS),
            "Query": query,
            "Codegen": "atlasr22",
            # 'IgnoreLocalCache': True,
        }
    ]
}
```